# 排序

In [14]:
<T> void swap(T[] data, int i, int j) {
    if (i < 0 || j < 0 || i >= data.length || j >= data.length) {
        throw new IllegalArgumentException();
    }
    T temp = data[i];
    data[i] = data[j];
    data[j] = temp;
}

In [15]:
<T> void print(T[] data) {
    for (int i = 0; i < data.length; i++) {
        System.out.printf("%s ", data[i].toString());
    }
    System.out.println();
}

## 选择排序

选择排序将待排序数组分为两个部分：位于数组前部的有序部分和位于数组后方的无序部分。每次循环时，选择无序部分的最值，将其与无需部分的第一个值交换，放入有序部分，直到无需部分为空。

In [10]:
<T extends Comparable> int findMin(T[] data, int start) {
    int index = start;
    T min = data[start];
    for (int i = start + 1; i < data.length; i++) {
        if (min.compareTo(data[i]) > 0) {
            min = data[i];
            index = i;
        }
    }
    return index;
}

In [12]:
<T extends Comparable> void sort(T[] data) {
    for (int i = 0; i < data.length; i++) {
        int index = findMin(data, i);
        swap(data, i, index);
    }
}

In [18]:
Integer[] data = new Integer[] {9, 8, 7, 6, 5, 4, 3, 2, 1, 0};
sort(data);
print(data);

0 1 2 3 4 5 6 7 8 9 
